In [20]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('Heart_Disease_Prediction.csv')

data.columns

Index(['Age', 'Sex', 'Chest pain type', 'BP', 'Cholesterol', 'FBS over 120',
       'EKG results', 'Max HR', 'Exercise angina', 'ST depression',
       'Slope of ST', 'Number of vessels fluro', 'Thallium', 'Heart Disease'],
      dtype='object')

In [21]:
data = data.drop(['ST depression','Slope of ST'], axis=1)
data = pd.get_dummies(data, drop_first=True)
data.head() 

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,Number of vessels fluro,Thallium,Heart Disease_Presence
0,70,1,4,130,322,0,2,109,0,3,3,True
1,67,0,3,115,564,0,2,160,0,0,7,False
2,57,1,2,124,261,0,0,141,0,0,7,True
3,64,1,4,128,263,0,0,105,1,1,7,False
4,74,0,2,120,269,0,2,121,1,1,3,False


In [22]:
X = data.drop(data.columns[-1], axis=1)
y = data[data.columns[-1]]
le = LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
print("Accuracy: ",model.evaluate(X_test, y_test))
model.save('heart_disease_model.tflite')

Epoch 1/50
7/7 [==============================] - 2s 52ms/step - loss: 12.0917 - accuracy: 0.4306 - val_loss: 6.4395 - val_accuracy: 0.3148
Epoch 2/50
7/7 [==============================] - 0s 10ms/step - loss: 5.4780 - accuracy: 0.3843 - val_loss: 5.7635 - val_accuracy: 0.5000
Epoch 3/50
7/7 [==============================] - 0s 11ms/step - loss: 5.3001 - accuracy: 0.4583 - val_loss: 4.2156 - val_accuracy: 0.3148
Epoch 4/50
7/7 [==============================] - 0s 10ms/step - loss: 3.6436 - accuracy: 0.3796 - val_loss: 4.2064 - val_accuracy: 0.3148
Epoch 5/50
7/7 [==============================] - 0s 10ms/step - loss: 2.9604 - accuracy: 0.3981 - val_loss: 2.5441 - val_accuracy: 0.3704
Epoch 6/50
7/7 [==============================] - 0s 10ms/step - loss: 1.9090 - accuracy: 0.4167 - val_loss: 1.6567 - val_accuracy: 0.3704
Epoch 7/50
7/7 [==============================] - 0s 11ms/step - loss: 1.0861 - accuracy: 0.4583 - val_loss: 0.8925 - val_accuracy: 0.6111
Epoch 8/50
7/7 [==========

INFO:tensorflow:Assets written to: heart_disease_model.tflite\assets


In [23]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('heart_disease_prediction_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ravik\AppData\Local\Temp\tmpkl9syub1\assets


INFO:tensorflow:Assets written to: C:\Users\ravik\AppData\Local\Temp\tmpkl9syub1\assets
